In [2]:
import os, sys
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
sys.path.append(BASE_DIR)

from src.models.Detection.Faster_RCNN import Faster_RCNN
from src.dataset.bdd_detetcion import BDD_Detection
from src.config.defaults import cfg
from pytorch_lightning import Trainer

from src.utils.DataLoaders import get_loader

In [3]:
bdd_train_params = {
    'cfg': cfg,
    'stage': 'train'
}

bdd_train = BDD_Detection(**bdd_train_params)

100%|██████████████████████████████████████████████████████████████████████████| 69863/69863 [00:07<00:00, 9672.24it/s]


In [4]:
bdd_val_params = {
    'cfg': cfg,
    'stage': 'val'
}

bdd_val = BDD_Detection(**bdd_val_params)

100%|█████████████████████████████████████████████████████████████████████████| 69863/69863 [00:06<00:00, 11528.73it/s]


In [5]:
train_dataloader_args = {
    'dataset': bdd_train,
    'batch_size': 32,
    'shuffle': True,
}
train_dataloader = get_loader(bdd_train, **train_dataloader_args)

val_dataloader_args = {'batch_size':1, 'shuffle':False}
val_dataloader = data.DataLoader(bdd_val, **val_dataloader_args)

NameError: name 'torch' is not defined

In [5]:
faster_rcnn_params = {
    'cfg': cfg,
    'num_classes': 7,
    'backbone': 'resnet101',
    'learning_rate': 1e-5,
    'weight_decay': 1e-3,
    'pretrained': True,
    'pretrained_backbone': True,
    'checkpoint_path': None,
}
model = Faster_RCNN(**faster_rcnn_params)

C:\Users\Khaliladib\anaconda3\envs\yoloc\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'backbone_name' and 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  warnings.warn(
C:\Users\Khaliladib\anaconda3\envs\yoloc\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
